In [1]:
#Assignment 2 - Building text processing models

# Group Details
# 01. IT17181402 D. G. Y. C. K. Alwis
# 02. IT17182188 R. M. M. G. Rathnayake
# 03. IT17175012 W. M. N. A. Fernando

#                 Topic               #
#         Spam Identification

# Dataset used in the assignment
# Kaggle SMS Spam Collection Dataset - Visit: https://github.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset- 

#             Introduction            #
#  Spam is a irrelevant or unsolicited messages sent over the Internet, 
#  typically to a large number of users, for the purposes of advertising, phishing, spreading malware.
#  To avoid this unnessary sms or emails we can build a model that identify the spams using specific keywords such as WIN, Bouns, Winner etc.
#  The code is such spam detection model that implement using python. enjoy!

# Spam vs Ham 
# "Ham" is e-mail that is not Spam. In other words, "non-spam", or "good mail". It should be considered a shorter, 
# snappier synonym for "non-spam". Its usage is particularly common among anti-spam software developers, 
# and not widely known elsewhere; in general it is probably better to use the term "non-spam", instead.

#Loading nessasary libraries

import nltk #importing natural language processing toolkit visit: https://www.nltk.org/ 
import numpy as np #importing numpy or python array object library visit: https://numpy.org/
import pandas as pd #importing pandas libarary, pyton data analysis library visit: https://pandas.pydata.org/
import matplotlib.pyplot as plt #importing matplotlib library, python data visualization library visit: https://matplotlib.org/
from sklearn.feature_extraction.text import CountVectorizer #importing count vectorizer visit: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import TfidfTransformer #importing tf-idf calculator visit: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier #importing classification algorithm visit: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
from sklearn.model_selection import train_test_split #importing data splitter visit: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.metrics import accuracy_score #importing accuracy score to determine model accuracy visit: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
from sklearn.metrics import confusion_matrix #importing confution matrix visit: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
from sklearn.metrics import classification_report # importing classification report to print precision, recall and f1 visit: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report 
from nltk.tokenize import word_tokenize #importing word tokenizer visit: https://www.nltk.org/api/nltk.tokenize.html
from nltk.corpus import stopwords #importing stopwords to remove stop words visit: https://pythonspot.com/nltk-stop-words/
from nltk.stem.wordnet import WordNetLemmatizer #importing word lemmatizer visit: https://www.nltk.org/_modules/nltk/stem/wordnet.html
from nltk.stem.porter import PorterStemmer #importing porterstemmer for stemming visit:  https://www.nltk.org/_modules/nltk/stem/porter.html
import string #importing string library to manipulate strings visit: https://docs.python.org/2/library/string.html

#download the nsessary stop word, punktuation, lemmatization libraries, 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#importing dataset using pandas dataframe
dataset = pd.read_csv("dataset.csv") #Please change the data file path when executing!

#print header or first 5 rows of the data
dataset.head()

[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


,Text,Class
0,dobmeos with hgh my energy level has gone up !...,spam
1,your prescription is ready . . oxwq s f e low ...,spam
2,get that new car 8434 people nowthe weather or...,spam
3,"await your response dear partner , we are a te...",spam
4,"coca cola , mbna america , nascar partner with...",spam


In [2]:
#Get the message part to clean process

#import stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

#remove all NaN ot NaT (Null) Values visit: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
dataset = dataset.dropna()

#Extarct the message from the dataset to process futher
message = dataset['Text']
message.head()

#turn all the letters in the dataframe to lowercase visit: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.lower.html
message = message.str.lower()

#remove any html tags in the dataset visit: https://www.w3schools.com/python/python_regex.asp
message = message.str.replace(r'<[^>]+>',' ')

#remove all special characters visit: https://www.w3schools.com/python/python_regex.asp
message = message.str.replace(r'[!"#$%&\'()*+,./:;<=>?@\^_`{|}~-]',' ')

#remove all numbers
message = message.str.replace(r'[0-9]',' ')

#remove single characters in the dataset
message = message.str.replace(r'[ ][a-z][ ]',' ')

#remove multiple spaces visit : https://www.w3schools.com/python/python_regex.asp
message = message.str.replace(r'[ ]{2,}', ' ')

#apply tokenization to the dataframe visit: https://www.nltk.org/api/nltk.tokenize.html
message = message.apply(nltk.word_tokenize)

#remove stop words from the dataframe
message = message.apply(lambda x: [word for word in x if word not in stop_words])

#lemmatize the words 
message = message.apply(lambda x:[lemmatizer.lemmatize(word) for word in x])

#setmming the words
message = message.apply(lambda x:[stemmer.stem(word) for word in x])

#print five rows of the processed data
message.head()

0    [dobmeo, hgh, energi, level, gone, stukm, intr...
1    [prescript, readi, oxwq, f, low, cost, prescri...
2    [get, new, car, peopl, nowth, weather, climat,...
3    [await, respons, dear, partner, team, govern, ...
4    [coca, cola, mbna, america, nascar, partner, o...
Name: Text, dtype: object

In [3]:
#From the cleaning process we get dataframe Series of lists
#Remove list and create the sentence again to calculate tf-idf 
build_messages = message.str.join(' ')
build_messages.head()

0    dobmeo hgh energi level gone stukm introduc do...
1    prescript readi oxwq f low cost prescript medi...
2    get new car peopl nowth weather climat particu...
3    await respons dear partner team govern offici ...
4    coca cola mbna america nascar partner otcbb im...
Name: Text, dtype: object

In [4]:
#Count vectorizer convert a collection of text documents to a matrix of token counts
countvc = CountVectorizer() #https://adataanalyst.com/scikit-learn/countvectorizer-sklearn-example/

#Fit data to the counter vectorizer and get document-term matrix
document_term_matrix = countvc.fit_transform(stri for stri in build_messages)

#print the document-term matrix
print(document_term_matrix[0:5,:])

#get the relevent term
print(countvc.get_feature_names()[9667])

  (0, 47011)	1
  (0, 48924)	1
  (0, 37405)	1
  (0, 38177)	1
  (0, 9001)	1
  (0, 19561)	1
  (0, 30929)	1
  (0, 43717)	1
  (0, 29682)	1
  (0, 8063)	1
  (0, 9640)	2
  (0, 20045)	1
  (0, 43057)	1
  (0, 21761)	1
  (0, 9124)	1
  (0, 38133)	1
  (0, 35313)	1
  (0, 40593)	1
  (0, 1184)	1
  (0, 28624)	1
  (0, 28590)	1
  (0, 42550)	1
  (0, 14415)	1
  (0, 41497)	1
  (0, 44626)	1
  :	:
  (4, 37511)	1
  (4, 40724)	2
  (4, 49508)	1
  (4, 40396)	1
  (4, 28079)	2
  (4, 728)	1
  (4, 40534)	1
  (4, 9287)	1
  (4, 871)	3
  (4, 22734)	5
  (4, 22764)	1
  (4, 14027)	1
  (4, 32094)	1
  (4, 33517)	1
  (4, 6899)	3
  (4, 18189)	1
  (4, 34801)	1
  (4, 40232)	1
  (4, 26229)	3
  (4, 37405)	1
  (4, 30929)	2
  (4, 26863)	2
  (4, 45101)	1
  (4, 4087)	1
  (4, 32389)	1
convict


In [5]:
#get the tf-idf from the document-term matrix

#create new object from tf-idf class
tfidf_transformer = TfidfTransformer()

#Input the document-term matrix and transfrom it to tf-idf
tfidf = tfidf_transformer.fit_transform(document_term_matrix)

#print tf-idf vales in the dataset
print(tfidf)

#get tf-idf value for word control
print (tfidf_transformer.idf_[countvc.vocabulary_['bitambr']])

  (0, 50818)	0.06696254378696476
  (0, 49894)	0.10055967195469231
  (0, 49816)	0.09869802882262106
  (0, 48924)	0.053503236914028425
  (0, 48129)	0.08227713600876241
  (0, 47011)	0.06273742979770587
  (0, 46241)	0.08384013051884252
  (0, 45101)	0.0319410785723564
  (0, 44844)	0.10397284981319102
  (0, 44626)	0.11963570438029639
  (0, 43717)	0.06891761144458002
  (0, 43163)	0.12491052537086636
  (0, 43057)	0.08968066277938404
  (0, 43056)	0.08044826193500586
  (0, 42550)	0.08968066277938404
  (0, 41497)	0.08110433250671219
  (0, 41396)	0.09057821238215277
  (0, 40593)	0.07675914327887542
  (0, 39980)	0.07496190279948364
  (0, 38177)	0.08968066277938404
  (0, 38133)	0.0995955784253898
  (0, 37686)	0.05753569500651184
  (0, 37655)	0.06566773249933254
  (0, 37405)	0.05353014345779247
  (0, 36906)	0.08884106119791826
  :	:
  (10879, 11095)	0.06171564628653134
  (10879, 10303)	0.04054969603488761
  (10879, 9640)	0.06865725688332348
  (10879, 9600)	0.030724673170547778
  (10879, 9571)	0.02373

In [6]:
#devide dataset into train set and test set
tfidf_trainset, tfidf_testset = train_test_split(tfidf,train_size=0.8)
tfidf_train_class, tfidf_test_class = train_test_split(dataset['Class'],train_size=0.8)

#Extract y or predicted ouput
expected_output = tfidf_test_class

In [7]:
#Creating Naive Bayes Model
NV_model = MultinomialNB()

#Train the model using the training sets
NV_model.fit(tfidf_trainset, tfidf_train_class)

#Predict the response for test dataset
test_prediction = NV_model.predict(tfidf_testset)

print('Accuracy of the Naive Bayes model: ')
print(NV_model.score(tfidf_testset,tfidf_test_class))


print('\nClassification Report')
print(classification_report(test_prediction, expected_output))

print('\nConfusion Matrix')
print(confusion_matrix(test_prediction, expected_output))

Accuracy of the Naive Bayes model: 
0.7320772058823529

Classification Report
              precision    recall  f1-score   support

         ham       0.99      0.74      0.84      2139
        spam       0.02      0.35      0.04        37

    accuracy                           0.73      2176
   macro avg       0.50      0.55      0.44      2176
weighted avg       0.97      0.73      0.83      2176


Confusion Matrix
[[1580  559]
 [  24   13]]


In [8]:
#Building Support Vector Machine 
svm_model = SVC(kernel = 'rbf', gamma = 'scale', C = 1)

#Fit data to the model to train
svm_model.fit(tfidf_trainset,tfidf_train_class)

#Predict the model using test dataset 
svm_predicted_output = svm_model.predict(tfidf_testset)

print('Accuracy of the model: ',svm_model.score(tfidf_testset, expected_output))
print('\nConfusion Matrix: ',confusion_matrix(svm_predicted_output, expected_output))
print('\nReport: ', classification_report(svm_predicted_output, expected_output))

Accuracy of the model:  0.7297794117647058

Confusion Matrix:  [[1582  566]
 [  22    6]]

Report:                precision    recall  f1-score   support

         ham       0.99      0.74      0.84      2148
        spam       0.01      0.21      0.02        28

    accuracy                           0.73      2176
   macro avg       0.50      0.48      0.43      2176
weighted avg       0.97      0.73      0.83      2176



In [9]:
#Building K-NN model 

#Creating K-NN model and set initial neighbors to 5
#After running several times with random k value we found that the 13 is most accurate
model = KNeighborsClassifier(n_neighbors=13)

#fit the data into model train
model.fit(tfidf_trainset,tfidf_train_class)

kn_predicted_output = model.predict(tfidf_testset)

print('Accuracy of the model: ',model.score(tfidf_testset,tfidf_test_class))
print('\nConfusion Matrix: ',confusion_matrix(kn_predicted_output,tfidf_test_class))
print('\nReport: ', classification_report(kn_predicted_output,tfidf_test_class))

Accuracy of the model:  0.7247242647058824

Confusion Matrix:  [[1569  564]
 [  35    8]]

Report:                precision    recall  f1-score   support

         ham       0.98      0.74      0.84      2133
        spam       0.01      0.19      0.03        43

    accuracy                           0.72      2176
   macro avg       0.50      0.46      0.43      2176
weighted avg       0.96      0.72      0.82      2176

